Tensorflow를 이용해서 CNN(Convolution Neural Network)으로 MNIST 데이터셋을 분류해 보도록 하겠습니다. 

MNIST 데이터 셋은 [Tensorflow에서 Mnist 데이터셋 살펴보기](https://comafire.github.io/posts/2016-python-tf-mnist)에서 다루었었으므로 참조하세요.

CNN은 Convolution Layer + Pooling Layer + Fully Connected Layer(DNN)의 조합이기에 CNN 구성 부분외에 Tensorflow를 통한 훈련과 평가는 [Tensorflow에서 DNN으로 Mnist 데이터셋 분류하기](https://comafire.github.io/posts/2016-python-tf-dnn-mnist)에서 다루었던 내용과 같으므로 해당 내용을 참조하세요. 

Tensorflow의 변수 생성을 함수를 통해 간편하게 하기 위해 변수 범위를 한정하는 방법은 [Tensorflow에서 variable_scope 사용하기](https://comafire.github.io/posts/2016-python-tf-dnn-mnist-with-vscope)에서 다루었으므로 참조하세요.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
%matplotlib inline

import tensorflow as tf
from tensorflow.contrib.learn.python.learn import datasets

In [2]:
mnist = datasets.mnist.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
with tf.name_scope("inputs"):
    x = tf.placeholder(tf.float32, shape=[None, 784])
    y = tf.placeholder(tf.float32, shape=[None, 10])
    keep_prob = tf.placeholder(tf.float32)

# convolution 층은 shape(filter의 사이즈와 갯수)를 입력받아 사용합니다.
# shape = [width, height, channel, filters]
# 필터 적용 후에 이미지 사이즈가 줄지 않도록 padding 은 SAME 을 사용합니다.
def conv_layer(x, shape):
    w = tf.get_variable(name="w", shape=shape, initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable(name="b", shape=[shape[3]], initializer=tf.contrib.layers.xavier_initializer())
    conv2d = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding="SAME")
    return tf.add(conv2d, b)

# pooling 층은 2x2 max pooling 을 사용하여 이미지를 반으로 줄입니다.
def pool_layer(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# fully connected 층은 DNN에서 사용했던 것과 동일합니다.
def fc_layer(x, shape):
    w = tf.get_variable(name="w", shape=shape, initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable(name="b", shape=[shape[1]], initializer=tf.contrib.layers.xavier_initializer())
    return tf.add(tf.matmul(x, w), b)

# CNN을 구성합니다.
with tf.name_scope("layers"):
    with tf.variable_scope("l1"):
        # CNN은 이미지를 입력으로 받기 때문에 Tensor를 
        # 28x28x1(width, height, channel)로 변형합니다.
        x_img = tf.reshape(x, [-1, 28, 28, 1])
        # filter 사이즈는 5x5x1 이고 32개를 사용합니다.
        conv1 = tf.nn.relu(conv_layer(x_img, [5, 5, 1, 32]))
        # 2x2 max pooling을 사용하기에 이미지는 14x14x32가 됩니다.
        pool1 = pool_layer(conv1)  
    with tf.variable_scope("l2"):
        # filter 사이즈는 5x5x32 이고 64개를 사용합니다.
        conv2 = tf.nn.relu(conv_layer(pool1, [5, 5, 32, 64]))
        # 2x2 max pooling을 사용하기에 이미지는 7x7x64가 됩니다.
        pool2 = pool_layer(conv2)
    with tf.variable_scope("l3"):
        # Convolution 층의 결과를 FC 층으로 보내기 위해 1차원으로 변형합니다.
        flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
        fc1 = tf.nn.relu(fc_layer(flat, [7 * 7 * 64, 1024]))
        dropout = tf.nn.dropout(fc1, keep_prob)
    with tf.variable_scope("l4"):
        fc2 = fc_layer(dropout, [1024, 10])

with tf.name_scope("optimize"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(fc2, y))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

with tf.name_scope("evaluate"):
    pred_y = tf.nn.softmax(fc2)
    corr_ys = tf.equal(tf.argmax(pred_y, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(corr_ys, tf.float32)) 

In [4]:
tb_cost = tf.summary.scalar("cost", cost)
tb_acc = tf.summary.scalar("acc", acc)
tb_merge = tf.summary.merge_all()

def mkdir(dir_path):
    if tf.gfile.Exists(dir_path):
        tf.gfile.DeleteRecursively(dir_path)
    tf.gfile.MakeDirs(dir_path)
        
with tf.Session() as sess:
    train_path = "../logs/tf-cnn-mnist-train"
    valid_path = "../logs/tf-cnn-mnist-valid"
    mkdir(train_path)
    mkdir(valid_path)

    train_writer = tf.train.SummaryWriter(train_path, sess.graph)
    valid_writer = tf.train.SummaryWriter(valid_path, sess.graph)

    for epoch in range(0, 10):    
        init = tf.global_variables_initializer()
        sess.run(init)    
        saver = tf.train.Saver()

        path_format = "../models/tf-cnn-mnist-%03d/"
        prev_model_path = path_format % (epoch - 1)
        prev_model_file = "%s%s" % (prev_model_path, "model")
        if tf.gfile.Exists(prev_model_path):
            saver.restore(sess, prev_model_file)

        for step in range(1000):
            train_batch_xs, train_batch_ys = mnist.train.next_batch(100)
            sess.run(optimizer, feed_dict={x: train_batch_xs, y: train_batch_ys, keep_prob: 0.5})
        
        # CNN 훈련후 cost 측정을 위해 전체 training 셋(55000개)을 사용할 경우 
        # 네트워크의 메모리 사용량이 많아져 컴퓨터 성능에 따라 결과가 안나올 수 있어
        # 마지막 training batch 데이터로 대신하였습니다.
        train_cost, train_acc, train_summary = sess.run([cost, acc, tb_merge], feed_dict={x: train_batch_xs, y: train_batch_ys, keep_prob: 1.0})
        train_writer.add_summary(train_summary, epoch)
        valid_cost, valid_acc, valid_summary = sess.run([cost, acc, tb_merge], feed_dict={x: mnist.validation.images, y: mnist.validation.labels, keep_prob: 1.0})
        valid_writer.add_summary(valid_summary, epoch)
        print "epoch: %d, train_cost: %s, train_acc: %s" % (epoch, train_cost, train_acc)
        print "epoch: %d, valid_cost: %s, valid_acc: %s" % (epoch, valid_cost, valid_acc)

        curr_model_path = path_format % epoch
        curr_model_file = "%s%s" % (curr_model_path, "model")
        mkdir(curr_model_path)
        saver.save(sess, curr_model_file)

    train_writer.close()
    valid_writer.close()

epoch: 0, train_cost: 0.056499, train_acc: 0.99
epoch: 0, valid_cost: 0.0418991, valid_acc: 0.987
epoch: 1, train_cost: 0.00947029, train_acc: 1.0
epoch: 1, valid_cost: 0.0332518, valid_acc: 0.989
epoch: 2, train_cost: 0.00461073, train_acc: 1.0
epoch: 2, valid_cost: 0.0324508, valid_acc: 0.9908
epoch: 3, train_cost: 0.0156652, train_acc: 0.99
epoch: 3, valid_cost: 0.0300711, valid_acc: 0.9922
epoch: 4, train_cost: 0.00332535, train_acc: 1.0
epoch: 4, valid_cost: 0.0284764, valid_acc: 0.9932
epoch: 5, train_cost: 0.00145708, train_acc: 1.0
epoch: 5, valid_cost: 0.0290569, valid_acc: 0.9934
epoch: 6, train_cost: 0.000389222, train_acc: 1.0
epoch: 6, valid_cost: 0.0323693, valid_acc: 0.9914
epoch: 7, train_cost: 7.62086e-05, train_acc: 1.0
epoch: 7, valid_cost: 0.0298904, valid_acc: 0.992
epoch: 8, train_cost: 0.000791616, train_acc: 1.0
epoch: 8, valid_cost: 0.0312531, valid_acc: 0.9922
epoch: 9, train_cost: 6.24243e-05, train_acc: 1.0
epoch: 9, valid_cost: 0.03711, valid_acc: 0.993


In [5]:
# 훈련 후 저장된 모델을 통한 테스트 데이터셋에서 99% 이상의 정확도를 확인할 수 있습니다.
with tf.Session() as sess:
    for epoch in range(10):
        init = tf.global_variables_initializer()
        sess.run(init)

        saver = tf.train.Saver()
        
        path_format = "../models/tf-cnn-mnist-%03d/"
        curr_model_path = path_format % epoch
        curr_model_file = "%s%s" % (curr_model_path, "model")
        saver.restore(sess, curr_model_file)

        test_acc = sess.run(acc, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
        print "epoch: %d, test acc: %s" % (epoch, test_acc)

epoch: 0, test acc: 0.9888
epoch: 1, test acc: 0.9902
epoch: 2, test acc: 0.9932
epoch: 3, test acc: 0.9924
epoch: 4, test acc: 0.9929
epoch: 5, test acc: 0.9924
epoch: 6, test acc: 0.9922
epoch: 7, test acc: 0.9927
epoch: 8, test acc: 0.9927
epoch: 9, test acc: 0.9934
